In [11]:
from emotion_recog.DeepFaceModel import DeepFaceModel
import cv2
import os
import numpy as np
from tqdm import tqdm

In [2]:
vectors = []
info = []
model = DeepFaceModel()

In [9]:
base = 'images/validation/'
for dirs in os.listdir(base):
    for ims in tqdm(os.listdir(base + dirs)):
        img = cv2.imread(base + dirs + '/' + ims)
        vectors.append(model.predict_vector(img, localize=False))
        info.append([ims, dirs])

100%|██████████| 111/111 [00:06<00:00, 18.00it/s]


In [12]:
vectors = np.array(vectors)
info = np.array(info)
np.save('emotion_vectors.npy', vectors)
np.save('emotion_vector_info.npy', info)